In [1]:
# Standard library
import warnings
import math

# Third party
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
%matplotlib inline

from scipy.io import wavfile

## Load dataset

In [2]:
df = pd.read_csv(f"../data/VTech/_VTech_merged.csv")

## Dataset information

In [3]:
print(f"Number of workers: {len(df['worker'].unique())}")
for worker, worker_df in df.groupby("worker"):
    print(f"\tWorker {worker}: {len(worker_df['day'].unique())} days"
          f", {len(worker_df['trial'].unique())} trials"
          f", {worker_df['mode'].diff().abs().sum():3.0f} transitions"
          f", {len(worker_df):10,} samples")
print(f"Total number of transitions: {df['mode'].diff().abs().sum()}")


Number of workers: 4
	Worker 1: 1 days, 5 trials,  28 transitions,  1,448,215 samples
	Worker 2: 3 days, 4 trials,  68 transitions,  1,351,866 samples
	Worker 3: 1 days, 2 trials,  59 transitions,    859,697 samples
	Worker 4: 1 days, 6 trials, 110 transitions,  1,646,663 samples
Total number of transitions: 266.0


In [23]:
df.loc[:, ["orientation_T8_q0", "orientation_T8_q1", "orientation_T8_q2", "orientation_T8_q3", "mode"] ]

Index(['Unnamed: 0', 'worker', 'day', 'trial', 'time', 'orientation_T8_q0',
       'orientation_T8_q1', 'orientation_T8_q2', 'orientation_T8_q3',
       'orientation_RightUpper_q0', 'orientation_RightUpper_q1',
       'orientation_RightUpper_q2', 'orientation_RightUpper_q3',
       'orientation_LeftUpper_q0', 'orientation_LeftUpper_q1',
       'orientation_LeftUpper_q2', 'orientation_LeftUpper_q3', 'position_T8_x',
       'position_T8_y', 'position_T8_z', 'position_RightUpper_x',
       'position_RightUpper_y', 'position_RightUpper_z',
       'position_LeftUpper_x', 'position_LeftUpper_y', 'position_LeftUpper_z',
       'velocity_T8_x', 'velocity_T8_y', 'velocity_T8_z',
       'velocity_RightUpper_x', 'velocity_RightUpper_y',
       'velocity_RightUpper_z', 'velocity_LeftUpper_x', 'velocity_LeftUpper_y',
       'velocity_LeftUpper_z', 'acceleration_T8_x', 'acceleration_T8_y',
       'acceleration_T8_z', 'acceleration_RightUpper_x',
       'acceleration_RightUpper_y', 'acceleration_Righ

## Building the TF Dataset

Stack consecutive `WINDOW_SIZE` samples with a sliding window, use the least sample for the target variable.
Do not overlap different workers/trials.
The resulting samples in the dataset are: `(WINDOW_SIZE, N_FEATURES) -> (1,)`

In [24]:
columns = ["orientation_T8_q0", "orientation_T8_q1", "orientation_T8_q2", "orientation_T8_q3", "mode"]

WINDOW_SIZE = 10
    
def make_windowed_dataset(ds, window_size, shift=1):
    windows = ds.window(window_size, shift=shift)

    def sub_to_batch(sub):
        return sub.batch(window_size, drop_remainder=True)

    def add_labels(batch):
        return batch[:, :-1], batch[-1, -1]
        #return batch[:, :-1], batch[0, -1]
    
    windows = windows.flat_map(sub_to_batch)
    return windows.map(add_labels)


ds = None
for trial_id, trial_data in df.groupby(["worker", "trial"]):
    ds_trial = make_windowed_dataset(
        tf.data.Dataset.from_tensor_slices(trial_data.loc[:,columns].values),
        window_size=WINDOW_SIZE
    )
    if ds is None:
        ds = ds_trial
    else:
        ds = ds.concatenate(ds_trial)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

## Model 

In [33]:
normalization = tf.keras.layers.experimental.preprocessing.Normalization()
normalization.adapt(ds.take(100000).map(lambda x, y: x))


model = keras.Sequential(
    [
        keras.layers.Input(shape=(WINDOW_SIZE, len(columns)-1)),
        normalization,
        keras.layers.Flatten(),
        keras.layers.Dense(16, activation="relu", name="layer1"),
        keras.layers.Dense(4, activation="relu", name="layer2"),
        keras.layers.Dense(1, activation= "sigmoid" ),
    ]
)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_5 (Normalizati (None, 10, 4)             9         
_________________________________________________________________
flatten_4 (Flatten)          (None, 40)                0         
_________________________________________________________________
layer1 (Dense)               (None, 16)                656       
_________________________________________________________________
layer2 (Dense)               (None, 4)                 68        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 738
Trainable params: 729
Non-trainable params: 9
_________________________________________________________________


In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy, 
              metrics=['accuracy'])
model.fit(ds.shuffle(buffer_size=2**18).batch(128), epochs=5)

Epoch 1/5
41456/41456 [==============================] - 886s 21ms/step - loss: 0.3427 - accuracy: 0.8421
Epoch 2/5
41456/41456 [==============================] - 890s 21ms/step - loss: 0.3423 - accuracy: 0.8466
Epoch 3/5
10013/41456 [======>.......................] - ETA: 11:42 - loss: 0.2636 - accuracy: 0.8874

KeyboardInterrupt: 

In [30]:
ds.

AttributeError: 'ConcatenateDataset' object has no attribute 'shape'

In [ ]:
y = np.array([y for _, y in ds])
ypred = model.predict(ds.batch(128)).flatten()
yhat = np.asarray(ypred > 0.5, dtype=np.float)
plt.plot(p[:4000])

In [ ]:
plt.plot(y[:100000])
plt.plot(yhat[:100000])

In [ ]:
np.sum(y == yhat) / len(y)

In [ ]:
wav = np.vstack((y, ypred, yhat)).T
wavfile.write("vtech_preds_late.wav", 25, wav)